## Titanic Dataset from Kaggle using Tensorflow 2
- Ref : https://towardsdatascience.com/the-googles-7-steps-of-machine-learning-in-practice-a-tensorflow-example-for-structured-data-96ccbb707d77

In [1]:
%matplotlib inline

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [19]:
import tensorflow as tf 
from tensorflow.keras import models, layers

## Gathering data

In [12]:
# Read data
df_raw = pd.read_csv('data/train.csv')

In [14]:
# Split data
df_train_raw, df_test_raw = train_test_split(df_raw, test_size=0.2)

## Preparing data
- Exploratory data analysis (EDA)
- Data Preprocessing

In [15]:
# Replace missing age data with average value of different Pclass group
def impute_age(cols):
    Age, Pclass = cols[0], cols[1]
    
    if pd.isnull(Age):
        if Pclass == 1:
            return 37
        elif Pclass == 2:
            return 29
        else:
            return 24
    else:
        return Age

In [16]:
def preprocessing(df):
    dfresult = pd.DataFrame()

    # Pclass -> one-hot encoding
    pclass = pd.get_dummies(df['Pclass'])
    pclass.columns = ['Pclass_' + str(x) for x in pclass.columns]
    dfresult = pd.concat([dfresult, pclass], axis=1)

    # Sex -> one-hot encoding
    sex = pd.get_dummies(df['Sex'])
    dfresult = pd.concat([dfresult, sex], axis=1)

    # Age -> replace with imputation
    dfresult['Age'] = df[['Age', 'Pclass']].apply(impute_age, axis=1)

    # Others
    dfresult['SibSp'] = df['SibSp']
    dfresult['Parch'] = df['Parch']
    dfresult['Fare'] = df['Fare']

    # Embarked -> one-hot encoding
    embarked = pd.get_dummies(df['Embarked'], dummy_na=True)
    embarked.columns = ['Embarked_' + str(x) for x in embarked.columns]
    dfresult = pd.concat([dfresult, embarked], axis=1)

    return dfresult

In [22]:
# Preprocessing (Train)
x_train = np.asarray(preprocessing(df_train_raw))
y_train = np.asarray(df_train_raw['Survived'].values)
print("x_train.shape =", x_train.shape )

x_train.shape = (712, 13)


In [23]:
# Preprocessing (Test)
x_test = np.asarray(preprocessing(df_test_raw))
y_test = np.asarray(df_test_raw['Survived'].values)
print("x_test.shape =", x_test.shape )

x_test.shape = (179, 13)


## Choose a model

In [20]:
tf.keras.backend.clear_session()

In [21]:
# Create Model
model = models.Sequential()

model.add(layers.Dense(10, activation='relu', input_shape=(13,)))
model.add(layers.Dense(20, activation='relu' ))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                140       
_________________________________________________________________
dense_1 (Dense)              (None, 20)                220       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
Total params: 381
Trainable params: 381
Non-trainable params: 0
_________________________________________________________________


## Training

In [24]:
# Compile
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train,y_train,
                    batch_size= 64,
                    epochs= 30,
                    validation_split=0.2)

## Model Evaluation

In [28]:
model.evaluate(x_test, y_test)

6/6 [==============================] - 0s 3ms/step - loss: 0.5682 - accuracy: 0.7207


[0.5682016015052795, 0.7206704020500183]

## Hyperparameter tuning

In [29]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [31]:
from tensorboard.plugins.hparams import api as hp

In [37]:
# Hyperparameter Configuration
HP_NUM_UNITS_ONE = hp.HParam('num_units_one', hp.Discrete([5, 10, 20]))
HP_NUM_UNITS_TWO = hp.HParam('num_units_two', hp.Discrete([10, 20, 40]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(hparams=[HP_NUM_UINTS_ONE, HP_NUM_UINTS_TWO, HP_OPTIMIZER],
                      metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')])

In [38]:
# Adapt TensorFlow runs to log hyperparameters and metrics
def train_test_model(hparams):
    # create model
    model = models.Sequential()

    model.add(layers.Dense(hparams[HP_NUM_UNITS_ONE], activation='relu', input_shape=(13,)))
    model.add(layers.Dense(hparams[HP_NUM_UNITS_TWO], activation='relu' ))
    model.add(layers.Dense(1, activation='sigmoid'))

    # compile
    model.compile(optimizer=hparams[HP_OPTIMIZER],
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # train
    history = model.fit(x_train, y_train,
                        batch_size=64, epochs=30, validation_split=0.2)

    # evalaution
    _, acc = model.evaluate(x_test, y_test)

    return acc

In [39]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [40]:
# Grid Search
session_num = 0
for num_units_one in HP_NUM_UNITS_ONE.domain.values:
    for num_units_two in HP_NUM_UNITS_TWO.domain.values:
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {HP_NUM_UNITS_ONE: num_units_one,
                       HP_NUM_UNITS_TWO: num_units_two,
                       HP_OPTIMIZER: optimizer,}

            run_name = "run-%d" % session_num
            print('>> Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})

            run('logs/hparam_tuning/' + run_name, hparams)

            session_num += 1

>> Starting trial: run-0
{'num_units_one': 5, 'num_units_two': 10, 'optimizer': 'adam'}
Epoch 1/30
9/9 [==============================] - 0s 29ms/step - loss: 2.7361 - accuracy: 0.3779 - val_loss: 2.3223 - val_accuracy: 0.4336
Epoch 2/30
9/9 [==============================] - 0s 4ms/step - loss: 2.3169 - accuracy: 0.3779 - val_loss: 1.9564 - val_accuracy: 0.4266
Epoch 3/30
9/9 [==============================] - 0s 4ms/step - loss: 1.9449 - accuracy: 0.3743 - val_loss: 1.6282 - val_accuracy: 0.4126
Epoch 4/30
9/9 [==============================] - 0s 4ms/step - loss: 1.6040 - accuracy: 0.3726 - val_loss: 1.3467 - val_accuracy: 0.4196
Epoch 5/30
9/9 [==============================] - 0s 4ms/step - loss: 1.3256 - accuracy: 0.3708 - val_loss: 1.1132 - val_accuracy: 0.4196
Epoch 6/30
9/9 [==============================] - 0s 4ms/step - loss: 1.0861 - accuracy: 0.3726 - val_loss: 0.9331 - val_accuracy: 0.4126
Epoch 7/30
9/9 [==============================] - 0s 4ms/step - loss: 0.9135 - accu

9/9 [==============================] - 0s 4ms/step - loss: 0.6300 - accuracy: 0.6608 - val_loss: 0.6326 - val_accuracy: 0.6503
Epoch 29/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6330 - accuracy: 0.6573 - val_loss: 0.6345 - val_accuracy: 0.6503
Epoch 30/30
6/6 [==============================] - 0s 929us/step - loss: 0.5803 - accuracy: 0.7207
>> Starting trial: run-2
{'num_units_one': 5, 'num_units_two': 20, 'optimizer': 'adam'}
Epoch 1/30
9/9 [==============================] - 0s 11ms/step - loss: 1.1682 - accuracy: 0.6661 - val_loss: 1.1539 - val_accuracy: 0.6364
Epoch 2/30
9/9 [==============================] - 0s 4ms/step - loss: 0.9825 - accuracy: 0.6714 - val_loss: 0.9738 - val_accuracy: 0.6294
Epoch 3/30
9/9 [==============================] - 0s 4ms/step - loss: 0.8565 - accuracy: 0.6696 - val_loss: 0.8339 - val_accuracy: 0.6364
Epoch 4/30
9/9 [==============================] - 0s 4ms/step - loss: 0.7500 - accuracy: 0.6696 - val_loss: 0.7346 - val_accuracy: 0

9/9 [==============================] - 0s 4ms/step - loss: 0.6244 - accuracy: 0.6731 - val_loss: 0.6587 - val_accuracy: 0.6643
Epoch 25/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6348 - accuracy: 0.6626 - val_loss: 0.6381 - val_accuracy: 0.6783
Epoch 26/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6333 - accuracy: 0.6555 - val_loss: 0.6480 - val_accuracy: 0.6434
Epoch 27/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6368 - accuracy: 0.6819 - val_loss: 0.6417 - val_accuracy: 0.6434
Epoch 28/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6379 - accuracy: 0.6696 - val_loss: 0.6383 - val_accuracy: 0.6783
Epoch 29/30
9/9 [==============================] - 0s 5ms/step - loss: 0.6558 - accuracy: 0.6661 - val_loss: 0.6694 - val_accuracy: 0.6643
Epoch 30/30
6/6 [==============================] - 0s 992us/step - loss: 0.6183 - accuracy: 0.7151
>> Starting trial: run-4
{'num_units_one': 5, 'num_units_two': 40, 'optimizer':

9/9 [==============================] - 0s 4ms/step - loss: 0.6231 - accuracy: 0.6678 - val_loss: 0.6182 - val_accuracy: 0.6503
Epoch 21/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6220 - accuracy: 0.6784 - val_loss: 0.6162 - val_accuracy: 0.6713
Epoch 22/30
9/9 [==============================] - 0s 5ms/step - loss: 0.6208 - accuracy: 0.6837 - val_loss: 0.6179 - val_accuracy: 0.6643
Epoch 23/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6231 - accuracy: 0.6766 - val_loss: 0.6163 - val_accuracy: 0.6713
Epoch 24/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6225 - accuracy: 0.6731 - val_loss: 0.6209 - val_accuracy: 0.6573
Epoch 25/30
9/9 [==============================] - 0s 5ms/step - loss: 0.6201 - accuracy: 0.6837 - val_loss: 0.6164 - val_accuracy: 0.6643
Epoch 26/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6203 - accuracy: 0.6784 - val_loss: 0.6206 - val_accuracy: 0.6573
Epoch 27/30
9/9 [======================

9/9 [==============================] - 0s 5ms/step - loss: 0.6378 - accuracy: 0.6573 - val_loss: 0.6232 - val_accuracy: 0.6434
Epoch 17/30
9/9 [==============================] - 0s 5ms/step - loss: 0.6491 - accuracy: 0.6696 - val_loss: 0.6309 - val_accuracy: 0.6643
Epoch 18/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6324 - accuracy: 0.6731 - val_loss: 0.7037 - val_accuracy: 0.5664
Epoch 19/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6573 - accuracy: 0.6274 - val_loss: 0.6309 - val_accuracy: 0.6503
Epoch 20/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6478 - accuracy: 0.6608 - val_loss: 0.6410 - val_accuracy: 0.6503
Epoch 21/30
9/9 [==============================] - 0s 5ms/step - loss: 0.6480 - accuracy: 0.6450 - val_loss: 0.6337 - val_accuracy: 0.6573
Epoch 22/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6391 - accuracy: 0.6432 - val_loss: 0.6233 - val_accuracy: 0.6573
Epoch 23/30
9/9 [======================

9/9 [==============================] - 0s 4ms/step - loss: 0.6513 - accuracy: 0.6678 - val_loss: 0.6248 - val_accuracy: 0.6643
Epoch 13/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6127 - accuracy: 0.6889 - val_loss: 0.6371 - val_accuracy: 0.6713
Epoch 14/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6202 - accuracy: 0.6784 - val_loss: 0.6064 - val_accuracy: 0.6573
Epoch 15/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6098 - accuracy: 0.6907 - val_loss: 0.6355 - val_accuracy: 0.6434
Epoch 16/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6168 - accuracy: 0.6889 - val_loss: 0.6075 - val_accuracy: 0.6573
Epoch 17/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6390 - accuracy: 0.6591 - val_loss: 0.6152 - val_accuracy: 0.6713
Epoch 18/30
9/9 [==============================] - 0s 5ms/step - loss: 0.6269 - accuracy: 0.6749 - val_loss: 0.6331 - val_accuracy: 0.6573
Epoch 19/30
9/9 [======================

9/9 [==============================] - 0s 4ms/step - loss: 0.6959 - accuracy: 0.6520 - val_loss: 0.6398 - val_accuracy: 0.5734
Epoch 9/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6366 - accuracy: 0.6380 - val_loss: 0.6105 - val_accuracy: 0.6573
Epoch 10/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6234 - accuracy: 0.6555 - val_loss: 0.6133 - val_accuracy: 0.6643
Epoch 11/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6311 - accuracy: 0.6801 - val_loss: 0.6002 - val_accuracy: 0.6643
Epoch 12/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6846 - accuracy: 0.6432 - val_loss: 0.5983 - val_accuracy: 0.6713
Epoch 13/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6096 - accuracy: 0.6766 - val_loss: 0.5930 - val_accuracy: 0.6573
Epoch 14/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6269 - accuracy: 0.6538 - val_loss: 0.5954 - val_accuracy: 0.6643
Epoch 15/30
9/9 [=======================

9/9 [==============================] - 0s 4ms/step - loss: 0.6266 - accuracy: 0.6766 - val_loss: 0.6236 - val_accuracy: 0.6643
Epoch 5/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6385 - accuracy: 0.6714 - val_loss: 0.6117 - val_accuracy: 0.6573
Epoch 6/30
9/9 [==============================] - 0s 4ms/step - loss: 0.7099 - accuracy: 0.6186 - val_loss: 0.6882 - val_accuracy: 0.6294
Epoch 7/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6306 - accuracy: 0.6837 - val_loss: 0.7364 - val_accuracy: 0.5664
Epoch 8/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6376 - accuracy: 0.6661 - val_loss: 0.7695 - val_accuracy: 0.6224
Epoch 9/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6757 - accuracy: 0.6538 - val_loss: 0.6290 - val_accuracy: 0.6783
Epoch 10/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6635 - accuracy: 0.6696 - val_loss: 0.8368 - val_accuracy: 0.5664
Epoch 11/30
9/9 [===========================

Epoch 1/30
9/9 [==============================] - 0s 11ms/step - loss: 1.4752 - accuracy: 0.5764 - val_loss: 0.6576 - val_accuracy: 0.6224
Epoch 2/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6250 - accuracy: 0.6801 - val_loss: 0.6285 - val_accuracy: 0.6503
Epoch 3/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6243 - accuracy: 0.6837 - val_loss: 0.6284 - val_accuracy: 0.6503
Epoch 4/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6175 - accuracy: 0.6889 - val_loss: 0.6376 - val_accuracy: 0.6643
Epoch 5/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6147 - accuracy: 0.6889 - val_loss: 0.6367 - val_accuracy: 0.6643
Epoch 6/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6114 - accuracy: 0.6995 - val_loss: 0.6155 - val_accuracy: 0.6643
Epoch 7/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6098 - accuracy: 0.6889 - val_loss: 0.7153 - val_accuracy: 0.5664
Epoch 8/30
9/9 [=================

Epoch 29/30
9/9 [==============================] - 0s 4ms/step - loss: 0.5077 - accuracy: 0.7715 - val_loss: 0.5597 - val_accuracy: 0.7273
Epoch 30/30
6/6 [==============================] - 0s 949us/step - loss: 0.4703 - accuracy: 0.8212
>> Starting trial: run-17
{'num_units_one': 20, 'num_units_two': 40, 'optimizer': 'sgd'}
Epoch 1/30
9/9 [==============================] - 0s 11ms/step - loss: 1.1332 - accuracy: 0.5712 - val_loss: 0.6809 - val_accuracy: 0.6853
Epoch 2/30
9/9 [==============================] - 0s 5ms/step - loss: 0.7625 - accuracy: 0.6081 - val_loss: 0.6780 - val_accuracy: 0.6503
Epoch 3/30
9/9 [==============================] - 0s 4ms/step - loss: 0.7127 - accuracy: 0.6239 - val_loss: 0.6597 - val_accuracy: 0.6154
Epoch 4/30
9/9 [==============================] - 0s 4ms/step - loss: 0.7172 - accuracy: 0.6661 - val_loss: 0.6957 - val_accuracy: 0.5874
Epoch 5/30
9/9 [==============================] - 0s 4ms/step - loss: 0.7634 - accuracy: 0.6415 - val_loss: 0.8490 - val

## Prediction (and save model)

In [ ]:
# Prediction
model.predict(x_test[0:10])

In [ ]:
# Prediction (Class)
model.predict_classes(x_test[0:10])

In [ ]:
# Save model
model.save('data/keras_model.h5')